In [9]:
import numpy as np
from spicy import stats
import pandas as pd

In [ ]:
class_names = ["crime", "romance", "science fiction", "western"]

df = pd.read_csv("../datos/wiki_movie_plots_deduped_sample.csv")

In [11]:
def leerDatos(dimension):
    # nos quedamos con todos los tokens
    tokens = np.hstack(df["tokens"].apply(lambda x: x.split()).values)

    # contamos la frecuencia y nos quedamos con los 1000 más frecuentes
    unique_tokens = pd.Series(tokens).value_counts().index[:dimension].values

    # hacemos diccionario que mapea de 0 a 999
    unique_tokens_dict = dict(zip(unique_tokens, range(len(unique_tokens))))

    # incializamos matriz
    X = np.zeros((len(df), len(unique_tokens)), dtype=int)

    # llenamos matriz de conteo
    for i, row in df.iterrows():
        for token in row["tokens"].split():
            if unique_tokens_dict.get(token,False)!=False:
                X[i, unique_tokens_dict[token]] += 1
    return X

In [12]:

def divisionEntrenoYTesteo(dimension):

    train_indices = df[df["split"] == "train"].index
    test_indices = df[df["split"] == "test"].index
    
    X = leerDatos(dimension)

    X_train = X[train_indices]
    X_test = X[test_indices]

    y_train = df[df["split"] == "train"]["Genre"]
    y_test = df[df["split"] == "test"]["Genre"]

    # Mapeo de géneros a números
    genero_a_numero = {
        "crime": 0,
        "romance": 1,
        "science fiction": 2,
        "western": 3
    }

    # Convertir y_train a números
    y_train_numerico = np.array([genero_a_numero[genero] for genero in y_train]).reshape(320, 1)
    y_test_numerico = np.array([genero_a_numero[genero] for genero in y_test]).reshape(80, 1)
    
    return X_train, y_train_numerico, X_test, y_test_numerico


In [13]:
def datosConDistribucion(dimension):
    X_train, y_train_numerico, X_test, y_test_numerico = divisionEntrenoYTesteo(dimension)
    
    num_generos = 4  # Tienes 4 géneros
    num_peliculas_por_genero = 80  #  en train

    # Inicializar una lista para almacenar los índices intercalados
    indices_intercalados_train = []
    indices_intercalados_test = []

    # Generar los índices intercalados para obtener el patrón 0, 1, 2, 3, 0, 1, 2, 3, ...
    for i in range(num_peliculas_por_genero):  # Iterar sobre los géneros
        for j in range(num_generos):  # Iterar sobre las películas por género
            indices_intercalados_train.append((j * num_peliculas_por_genero) + i)
    
    num_peliculas_por_genero = 20 # en test
    for i in range(num_peliculas_por_genero):  # Iterar sobre los géneros
        for j in range(num_generos):  # Iterar sobre las películas por género
            indices_intercalados_test.append((j * num_peliculas_por_genero) + i)

    # Convertir la lista a arreglo numpy para usar en el reordenamiento
    indices_intercalados_train = np.array(indices_intercalados_train)
    indices_intercalados_test = np.array(indices_intercalados_test)

    # Reordenar y_train_numerico y matriz_conteo según los índices intercalados
    y_train_numerico_reordenada = y_train_numerico[indices_intercalados_train]
    X_train_reordenada = X_train[indices_intercalados_train]
    
    y_test_numerico_reordenada = y_train_numerico[indices_intercalados_test]
    X_test_reordenada = X_train[indices_intercalados_test]
    
    return X_train_reordenada, y_train_numerico_reordenada, X_test_reordenada, y_test_numerico_reordenada